# Quick start: using Jack models

## Prerequisites

Note: these commands need to be run in terminal from the root of Jack.

Download GloVe vectors:
> `sh data/GloVe/download.sh`

Download pretrained FastQA and DAM models:
> `wget -O fastqa.zip https://www.dropbox.com/s/qb796uljoqj0lvo/fastqa.zip?dl=1`

> `wget -O dam.zip http://data.neuralnoise.com/jack/natural_language_inference/dam.zip`

Prepare the model for use:
> `unzip fastqa.zip`

> `unzip dam.zip`

First, let's get the imports sorted:

In [2]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('..')    # change dir to Jack root

In [14]:
from jack import readers
from jack.core import QASetting
from jack.io.load import load_jack
from notebooks.prettyprint import QAPrettyPrint

## Usecase: QA 

Load the (previously downloaded) pretrained FastQA model:

In [4]:
fastqa_reader = readers.reader_from_file("./fastqa")

INFO:tensorflow:Restoring parameters from ./fastqa/model_module


Let's define a reading comprehension paragraph and a question:

In [5]:
paragraph = """It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. 
At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary."""

question = "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?"

We merge them into a single `QASetting` data structure. This structure requires a question and a list of supporting documents:

In [6]:
qa_setting = QASetting(question=question, support=[paragraph])

Feed the `qa_setting` (paragraph and the question) structure into the reader to get the answers:

In [7]:
answers = fastqa_reader([qa_setting])

The answer can be found here:

In [8]:
answers[0][0].text

'Saint Bernadette Soubirous'

...together with the answer span, which we use to highlight the answer in the text:

In [9]:
QAPrettyPrint(paragraph, answers[0][0].span)

...and the score of the answer:

In [10]:
answers[0][0].score

36.591003

## Usecase: Natural Language Inference (NLI) 

We first load a pretrained DAM [[1]](#ref1) model:

In [10]:
dam_reader = readers.reader_from_file("./dam")

INFO:tensorflow:Restoring parameters from ./dam/model_module


and next some Natural Language Inference examples from the SNLI corpus [[2]](#ref2):

In [11]:
premise = "A boy is jumping on skateboard in the middle of a red bridge."
hypothesis = "The boy does a skateboarding trick."

In the NLI case, the answer is a label among "entailment", "neutral", and "contradiction".

We can again use the same data structure `QASetting` as above to feed Jack entailment data. Let's define a new input example:

In [23]:
premise = "A wedding party is taking pictures."
hypothesis1 = "A group of people is celebrating."
snli_setting1 = QASetting(question=hypothesis1, support=[premise])

hypothesis2 = "A rock band is giving a concert."
snli_setting2 = QASetting(question=hypothesis2, support=[premise])

We generate preditions by calling the reader with these inp:

In [28]:
prediction = dam_reader([snli_setting1])
print(prediction[0][0].text, "\t", premise, hypothesis1, )

prediction = dam_reader([snli_setting2])
print(prediction[0][0].text, "\t", premise, hypothesis2)



entailment 	 A wedding party is taking pictures. A group of people is celebrating.
contradiction 	 A wedding party is taking pictures. A rock band is giving a concert.


and we can again also inspect the highest scoring prediction and its score:

## References

<a id='ref1'>[1]</a> Ankur Parikh, Oscar Täckström, Dipanjan Das, Jakob Uszkoreit . <a href='http://www.aclweb.org/anthology/D14-1162'>"A Decomposable Attention Model for Natural Language Inference."</a> Proceedings of the 2016 conference on empirical methods in natural language processing (EMNLP). 2016. 

<a id='ref2'>[2]</a> Samuel R. Bowman, Gabor Angeli, Christopher Potts, and Christopher D. Manning . A large annotated corpus for learning natural language inference. In Proceedings of the 2015 Conference on Empirical Methods in Natural Language Processing (EMNLP). 2015.